# การติดตั้งและเริ่มใช้งาน Hyperledger Fabric เบื้องต้น

*** ตัวอย่างนี้เป็นการติดตั้งและทดลองรัน Hyperledger Fabric version อย่างง่าย ซึ่งไม่เหมาะแ่กการใช้งานใน production environment ***

## Requirements

1. ติดตั้งโดยเครื่องมือ สคริปต์ และตัวอย่างที่ทาง official Hyperledger มีให้
2. ติดตั้งบนเครื่อง Stand alone เพียงเครื่องเดียว
3. ติดตั้งโดยใช้ Docker CE เป็นหลัก
4. ใช้ตัวอย่าง chaincode example_02 ที่ official มีให้

## System requirements

1. OS: Ubuntu 18.04 LTS (MacOS/Windows ก็สามารถติดตั้งได้เช่นกัน แต่ต้องใช้ทักษะ config Docker เพิ่มเติมเพื่อติดตั้ง)
2. CPU: ยิ่งเยอะยิ่งดี เพราะ Blockchain มี service หลาย service มาก รวมไปถึงใช้งาน database หลาย instance
3. RAM: อย่างน้อย 8 GB สำหรับการติดตั้งแบบเครื่องเดียว
4. Internet: ไม่เช่นนั้นจะไม่สามารถ run docker container ได้

## Step โดยรวมในตัวอย่างนี้ประกอบด้วย

0. Setup เครื่องสำหรับใช้งาน notebook นี้
1. Setup เครื่องสำหรับการติดตั้งและใช้งาน Hyperledger Fabric
2. Generate crypto materail ซึ่งเป็น artifact สำหรับการสร้าง blockchain network และ permissioned channel สำหรับการทำ transaction  ต่างๆ
3. deploy blockchain network และทดสอบด้วย end to end test ที่ตัวอย่างให้มา
4. terminate blockchain service 

สำหรับเครื่องที่ใช้ทดสอบตัวอย่างนี้มีสเปคดังนี้

In [1]:
lsb_release -a
lscpu | grep "Model name"
lsmem | grep "Total online memory"

No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 18.04.4 LTS
Release:	18.04
Codename:	bionic
Model name:          Intel(R) Core(TM) i7-8665U CPU @ 1.90GHz
Total online memory:    15.9G


## Step 0 Install & Setup environment สำหรับ notebook นี้

*** step นี้ให้เปิด terminal และ run แต่ละ command ใน terminal นะครับ ***

ติดตั้ง  [Bash kernel](https://github.com/takluyver/bash_kernel) สำหรับ Jupyter จะได้ run Ubuntu command ได้จากใน notebook

```shell
pip install bash_kernel
python -m bash_kernel.install
```

แก้ให้ไม่ต้องใส่ password สำหรับคำสั่ง sudo ด้วยการ edit file /etc/sudoers

```shell
sudo visudo
```

ใส่

```text
username     ALL=(ALL) NOPASSWD:ALL
```

เข้าไปในบรรทัดสุดท้ายและ save file /etc/sudoers

เปิด notebook นี้ด้วยการรันคำสั่งใน terminal

```shell
jupyter notebook
```

## Step 1 Setup เครื่องที่ใช้งาน

### ติดตั้ง docker engine และ docker-compose ต่างๆ

ในบทความนี้จะใช้ Docker Commutnity Edition (Docker CE) เป็นหลักนะครับ

In [2]:
sudo apt-get update

sudo apt-get install -y \
    apt-transport-https \
    ca-certificates \
    curl \
    gnupg-agent \
    software-properties-common \
    tree \
    zip unzip 
    
curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo apt-key add -

sudo add-apt-repository \
   "deb [arch=amd64] https://download.docker.com/linux/ubuntu \
   $(lsb_release -cs) \
   stable"
   
sudo apt-get update

sudo apt-get install docker-ce docker-ce-cli containerd.io docker-compose

Err:1 http://dl.google.com/linux/chrome/deb stable InRelease
  Could not resolve 'dl.google.com'
Err:2 https://download.docker.com/linux/ubuntu bionic InRelease                
  Could not resolve 'download.docker.com'
Err:3 http://packages.microsoft.com/repos/vscode stable InRelease              
  Could not resolve 'packages.microsoft.com'
Err:4 https://dl.winehq.org/wine-builds/ubuntu bionic InRelease                
  Could not resolve 'dl.winehq.org'
Err:5 https://repo.windscribe.com/ubuntu bionic InRelease                      
  Could not resolve 'repo.windscribe.com'
Err:6 http://th.archive.ubuntu.com/ubuntu bionic InRelease                     
  Could not resolve 'th.archive.ubuntu.com'
Err:7 http://ppa.launchpad.net/longsleep/golang-backports/ubuntu bionic InRelease
  Could not resolve 'ppa.launchpad.net'
Err:8 http://security.ubuntu.com/ubuntu bionic-security InRelease       
  Could not resolve 'security.ubuntu.com'
Err:9 http://th.archive.ubuntu.com/ubuntu bionic-updates 

W: Failed to fetch http://packages.microsoft.com/repos/vscode/dists/stable/InRelease  Could not resolve 'packages.microsoft.com'
W: Failed to fetch https://repo.windscribe.com/ubuntu/dists/bionic/InRelease  Could not resolve 'repo.windscribe.com'
W: Some index files failed to download. They have been ignored, or old ones used instead.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
docker-compose is already the newest version (1.17.1-2).
containerd.io is already the newest version (1.2.13-1).
docker-ce-cli is already the newest version (5:19.03.8~3-0~ubuntu-bionic).
docker-ce is already the newest version (5:19.03.8~3-0~ubuntu-bionic).
The following packages were automatically installed and are no longer required:
  gccgo-8 libgo13 linux-headers-5.3.0-28 linux-headers-5.3.0-28-generic
  linux-image-5.3.0-28-generic linux-modules-5.3.0-28-generic
  linux-modules-extra-5.3.0-28-generic pkg-config
Use 'sudo apt autoremove' to remove them.
0 

### ตั้งค่าให้ user สามารถใช้ docker ได้โดยไม่ต้องใช้ `sudo` ([Docker post installation](https://docs.docker.com/install/linux/linux-postinstall/))


In [3]:
sudo groupadd docker
sudo usermod -aG docker $USER

groupadd: group 'docker' already exists


## ทดสอบการทำงานของ docker

In [4]:
docker run hello-world

Unable to find image 'hello-world:latest' locally
docker: Error response from daemon: Get https://registry-1.docker.io/v2/: dial tcp: lookup registry-1.docker.io: no such host.
See 'docker run --help'.


: 125

## Step 2 Download Hyperledger Fabric binary files และ ตัวอย่าง

download file ที่จำเป็นในการใช้งาน โดยจะมีสิ่งที่ต้อง download 3 ประเภทหลัก คือ 

### 1. Hyperledger Fabric docker images
เป็น docker image ที่ pack core service ของ Hyperledger Fabric เอาไว้แล้ว ทำให้เราไม่ต้องติดตั้ง service เอง ทำให้เรา deploy blockchain network ได้คล่องตัวกว่า ซึ่งเราสามารถ custom config/parameter ต่าง ๆ ผ่านไฟล์ docker-compose.yaml ได้
### 2. Hyperledger Fabric samples 
เป็นตัวอย่าง config และ chaincode (smart contract) ที่ทางทีมงาน Hyperledger เตรีมไว้ให้ทดลองเล่น และสามารถนำไปประยุกต์ใช้กับ app ของเราได้
### 3. Hyperledger Fabric platform-specific binaries
เป็น tool สำหรับการสร้าง crypto material และเรียกใช้งาน blockchain service ต่างๆ

คำสั่งที่ใช้ download ทุกอย่างแบบปกติ คือ

```shell
curl -sSL http://bit.ly/2ysbOFE | bash -s
```

step นี้จะใช้เวลาค่อนข้างนาน เพราะต้อง download
1. Hyperledger Fabric docker images จำนวนมาก
2. Hyperledger Fabric samples หลายตัวอย่าง
3. Hyperledger Fabric platform-specific binaries ซึ่งใช้เวลา download ค่อนข้างนาน


เรามามารถ bypass ได้โดยการใส่ option

-d : bypass docker image download

-s : bypass fabric-samples repo clone

-b : bypass download of platform-specific binaries

ตัวอย่างการใช้งานเมื่อไม่ต้องการ download docker image และ samples

```shell
curl -sSL http://bit.ly/2ysbOFE | bash -s -- -d -s
```

ในตัวอย่างนี้จะทำการ download เฉพาะ binary file และตัวอย่างเท่านั้น ซึ่ง docker image ที่จำเป็นต้องใช้จะถูก pull ในขณะ deploy service อยู่แล้ว

In [5]:
curl -sSL http://bit.ly/2ysbOFE | bash -s -- -d

curl: (6) Could not resolve host: bit.ly


[ข้อมูลการติดตั้งเพิ่มเติม](https://hyperledger-fabric.readthedocs.io/en/release-1.4/install.html)

Set system path เพื่อให้ระบบมองเห็น binary

In [6]:
export PATH=$HOME/bin:$PATH

## Step 3 Deploy Hyperledger Fabric Network

change directory ไปที่ fabric-samples/first-network

In [7]:
cd ~/fabric-samples/test-network

ลองดูว่าใน script first network สามารถทำอะไรได้บ้าง

In [8]:
bash network.sh -h

Usage: 
  network.sh <Mode> [Flags]
    <Mode>
      - 'up' - bring up fabric orderer and peer nodes. No channel is created
      - 'up createChannel' - bring up fabric network with one channel
      - 'createChannel' - create and join a channel after the network is created
      - 'deployCC' - deploy the fabcar chaincode on the channel
      - 'down' - clear the network with docker-compose down
      - 'restart' - restart the network

    Flags:
    -ca <use CAs> -  create Certificate Authorities to generate the crypto material
    -c <channel name> - channel name to use (defaults to "mychannel")
    -s <dbtype> - the database backend to use: goleveldb (default) or couchdb
    -r <max retry> - CLI times out after certain number of attempts (defaults to 5)
    -d <delay> - delay duration in seconds (defaults to 3)
    -l <language> - the programming language of the chaincode to deploy: go (default), java, javascript, typescript
    -v <version>  - chaincode version. Must be a round num

: 1

deploy orderer, peer0.org1, peer0.org2 รอไว้สร้าง channel อีกที

In [10]:
# enter y to continue
bash network.sh up -i 2.1.0

Starting nodes with CLI timeout of '5' tries and CLI delay of '3' seconds and using database 'leveldb' 

Unable to find image 'hyperledger/fabric-tools:2.1.0' locally
2.1.0: Pulling from hyperledger/fabric-tools
aad63a933944: Pulling fs layer
c732a2540651: Pulling fs layer
7b2225181d6b: Pulling fs layer
c8dae7ec6990: Pulling fs layer
08684ee472f3: Pulling fs layer
da7709617ee6: Pulling fs layer
c414c3a96843: Pulling fs layer
e41f76c354cf: Pulling fs layer
08684ee472f3: Waiting
da7709617ee6: Waiting
c414c3a96843: Waiting
7b2225181d6b: Verifying Checksum
7b2225181d6b: Download complete
c732a2540651: Verifying Checksum
c732a2540651: Download complete
aad63a933944: Verifying Checksum
aad63a933944: Pull complete
c732a2540651: Pull complete
7b2225181d6b: Pull complete
08684ee472f3: Download complete
da7709617ee6: Verifying Checksum
da7709617ee6: Download complete
e41f76c354cf: Download complete
c414c3a96843: Verifying Checksum
c414c3a96843: Download complete
c8dae7ec6990: Verifying Checksum


ตรวจสอบการ deploy ว่าสำเร็จหรือไม่จาก log ที่ออกมา

ตรวจสอบ service ที่ถูก deploy ว่ามีอะไรบ้าง

In [11]:
docker ps

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES


## Step 3 Create Channel

สร้าง channel ชื่อ `getstartfabric`

In [59]:
# bash network.sh createChannel -c getstartfabric -i 2.1.0
bash network.sh createChannel -c getstartfabric -i 2.1.0

Creating channel 'getstartfabric'.

If network is not up, starting nodes with CLI timeout of '5' tries and CLI delay of '3' seconds and using database 'leveldb 

### Generating channel configuration transaction 'getstartfabric.tx' ###
+ configtxgen -profile TwoOrgsChannel -outputCreateChannelTx ./channel-artifacts/getstartfabric.tx -channelID getstartfabric
2020-05-08 08:51:04.772 +07 [common.tools.configtxgen] main -> INFO 001 Loading configuration
2020-05-08 08:51:04.789 +07 [common.tools.configtxgen.localconfig] Load -> INFO 002 Loaded configuration: /home/rattaphon/fabric-samples/test-network/configtx/configtx.yaml
2020-05-08 08:51:04.789 +07 [common.tools.configtxgen] doOutputChannelCreateTx -> INFO 003 Generating new channel configtx
2020-05-08 08:51:04.796 +07 [common.tools.configtxgen] func1 -> PANI 004 proto: Marshal called with nil
panic: proto: Marshal called with nil [recovered]
	panic: proto: Marshal called with nil

goroutine 1 [running]:
github.com/hyperledger/fabric/ven

: 1

## Step 4 Terminate Hyperledger Fabric Network

เมื่อต้องการหยุดการทำงานของ blockchain network ให้ทำการ shutdown และ clean docker container ต่างๆด้วยคำสั่งดังนี้

*** ระวัง คำสั่งนี้จะทำการ kill docker container ทั้งหมดในเครื่อง ****

In [15]:
echo "y" |
bash ./byfn.sh down -c getstartfabric

Stopping for channel 'getstartfabric' with CLI timeout of '10' seconds and CLI delay of '3' seconds
proceeding ...












Removing network net_byfn
Removing volume net_peer0.org3.example.com
Removing volume net_peer1.org3.example.com
Removing volume net_orderer2.example.com
Removing volume net_orderer.example.com
Removing volume net_peer0.org2.example.com
Removing volume net_peer0.org1.example.com
Removing volume net_peer1.org1.example.com
Removing volume net_peer1.org2.example.com
Removing volume net_orderer5.example.com
Removing volume net_orderer4.example.com
Removing volume net_orderer3.example.com
fbf88c3eb635
29a73802c5a4
3043228091dd
Untagged: dev-peer1.org2.example.com-mycc-1.0-26c2ef32838554aac4f7ad6f100aca865e87959c9a126e86d764c8d01f8346ab:latest
Deleted: sha256:71a46eb7b2062f663d835dba23b60e9c51b86933040dc9c06cb464e4e512da2c
Deleted: sha256:60f8796515107af6a31d75774f847ea819ea8f2946fbfa5a5066d472b3382514
Deleted: sha256:7ada5ddb7c1d18111ff31ded6e4722b8ae8b5381e96e78d76e

## สรุป

ทีนี้เราก็สามารถใช้เครื่องนี้เพื่อรัน Ubuntu command จาก notebook เพื่อใช้งาน Hyperledger Fabric ได้ และได้ลองทำการสร้าง crypto materials ซึ่งประกอบด้วย private key และ certificate จำนวนมาก ซึ่งจะเป็นองค์ประกอบหลักในการ deploy blockchain network เพื่อใช้งาน โดยในบทความถัดๆไปจะพาไปทำความรู้จักกับเจ้า crypto materials นี้แบบเข้มข้นสุดๆ นอกจากนี้เรายังได้ลองทำการ deploy chaincode ซึ่งเทียบเท่ากับ smart contract ของ blockchain อื่นๆ และทดลองทำการ รันแบบ end-2-end เพื่อทดสอบการทำงานระบบทั้งหมด และสุดท้าย เมื่อเราไม่ต้องการใช้งาน Hyperledger Fabric network ต่อเราก็สามารถปิดการทำงานและ clean สิิ่งที่เก็ดจากการ deploy ออก ซึ่งการ terminate ที่ถูกต้องจะมีประโยชน์อย่างมากในภายหลัง สำหรับบทความถัดไปจะเป็นเรื่องเกี่ยวกับการปรับแต่ง config ต่างๆเพื่อใช้งาน และทดลอง install และใช้งานตัวอย่าง chaincode แบบอื่นๆ
